### Manipulating Volumes

If we manipulate the image data, for example a crop or flip, we need to update the affine matrix as well. If not, the image geometry will be wrong and this could be dangerous for future use of the data.

Authors: David Atkinson

First version: 20 June 2021

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).
Copyright 2021 University College London.

This is software developed for the Collaborative Computational Project in Synergistic Reconstruction for Biomedical Imaging (http://www.ccpsynerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

In [ ]:
# Setup the working directory for the notebook
import notebook_setup
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('Geometry')

In [ ]:
# Import required packages
import nibabel 
import matplotlib.pyplot as plt
import numpy as np
import os
import sirf.Reg as Reg

In [ ]:
# Data for geometry notebooks when run is ./nifti/*.nii
data_path = os.getcwd()

In [ ]:
# Set numpy print options to print small numbers as zero etc.
np.set_printoptions(precision=3,suppress=True)

In [ ]:
%matplotlib widget

In [ ]:
def vdisp(ax, vol, A, falpha, frame, ocmap='gray'):
    # 3D Display of volume
    # sdisp(ax, s_ido, falpha, frame, ocmap='gray')
    #  ax      axes predefined
    #  vol     array of volume data
    #  A       4x4 affine matrix
    #  falpha  face alpha (0-1)
    #  frame   frame number (0-based)
    #  ocmap   colormap defaults to gray
    #
    # Calculates the vertices of pixels and uses to create a surface with transparency
    # falpha and intensities corresponding to pixwl values
    
    img = vol[:,:,frame]
    
    nrow = img.shape[0]
    ncol = img.shape[1]
    
    L = np.zeros((nrow+1, ncol+1))  # allocate memory
    P = np.zeros((nrow+1, ncol+1))  # +1 because this is for vertices
    H = np.zeros((nrow+1, ncol+1))
    
    for ir in range(0,nrow+1):
        for ic in range(0,ncol+1):
            # VLPH are LPH patient coordinates corresponding to
            # pixel vertices, which are at image coords -0.5, 0.5, 1.5, ...
            VLPH = np.matmul(A, np.array([ [ir-0.5], [ic-0.5], [frame], [1] ]))
            
            L[ir,ic] = VLPH[0] #  separate the components for surf plot
            P[ir,ic] = VLPH[1] 
            H[ir,ic] = VLPH[2] 
    
    scamap = plt.cm.ScalarMappable(cmap=ocmap)
    fcolors = scamap.to_rgba(img, alpha=falpha)
    ax.plot_surface(L, P, H, facecolors=fcolors, cmap=ocmap, linewidth=0, rcount=100, ccount=100)
    ax.set_xlabel('Left')
    ax.set_ylabel('Posterior')
    ax.set_zlabel('Head')

In [ ]:
# Read in data
# Get the affine matrix from NIfTI and convert to LPH
fpath  = os.path.join(data_path , 'nifti')
fn_cor = "OBJECT_phantom_T2W_TSE_Cor_14_1.nii" # Coronal volume, 30 slices
ffn = os.path.join(fpath, fn_cor)  # full file name


s_imd = Reg.ImageData(ffn)     # SIRF ImageData object
vol   = s_imd.as_array()     # SIRF array (the volume)

s_geom_info = s_imd.get_geometrical_info()
A_LPH = s_geom_info.get_index_to_physical_point_matrix()  # 4x4 affine matrix

print(A_LPH)
print(vol.shape)

In [ ]:
# Find the 3D coordinate of the offset point
Q = np.matmul(A_LPH,[0,0,0,1]) 
print(Q)

We are going to create a new volume from the central 200x200x20 region.

Will the spacing change?

Will the orientations change?

Will the offset change?


In [ ]:
# Calculate the image coordinates of the region we are going to extract.
# There might be more elegant ways of doing this, but Python has made a mess of 
# rounding and division in its various versions, so this is supposed to be clear

fov      = np.array([200, 200, 20])          # new field of view in pixels
center0b = np.floor(np.array(vol.shape) / 2.0) # 0-based coordinate of centre
hw = np.floor(fov/2.0)   # half width of new fov

lim = (center0b - hw).astype(int)  # Python ....

# Extract new volume from old
volnew = vol[lim[0]:lim[0]+fov[0], lim[1]:lim[1]+fov[1], lim[2]:lim[2]+fov[2]]
print(volnew.shape)


In [ ]:
# Calculate the new offset in 3D LPH
#  lim is in 0-based units
Qnew = np.matmul(A_LPH, [lim[0],lim[1],lim[2],1])


# The new A is the same as the old, excet for the updated offset
Anew = np.array([[A_LPH[0,0], A_LPH[0,1], A_LPH[0,2], Qnew[0]], 
                 [A_LPH[1,0], A_LPH[1,1], A_LPH[1,2], Qnew[1]],
                 [A_LPH[2,0], A_LPH[2,1], A_LPH[2,2], Qnew[2]],
                 [A_LPH[3,0], A_LPH[3,1], A_LPH[3,2], Qnew[3]] ])


print(A_LPH)
print(Anew)

In [ ]:
fig = plt.figure()          # Open figure and get 3D axes (can rotate with mouse)
ax  = plt.axes(projection='3d') 

vdisp(ax, volnew, Anew,  0.6, 10, 'gray')
vdisp(ax, vol,    A_LPH, 0.2, 15, 'gray')

The figure above shows that the cropped region comes correctly from the original.

Now lets look at an example of flipping the second dimension. 

Will the spacing change? 

Will the orientations change? 

Will the offset change?


In [ ]:
# The new offset will be at the position of the last voxel in the 2nd dimension in the original colume
Qnew = np.matmul(A_LPH, [0, vol.shape[1]-1, 0, 1])
print(Qnew)

In [ ]:
# The new A will use the updated offset and swap the sign of the 2nd column as this
# corresponds to the 2nd array index.
Anew = np.array([[A_LPH[0,0], -A_LPH[0,1], A_LPH[0,2], Qnew[0]], 
                 [A_LPH[1,0], -A_LPH[1,1], A_LPH[1,2], Qnew[1]],
                 [A_LPH[2,0], -A_LPH[2,1], A_LPH[2,2], Qnew[2]],
                 [A_LPH[3,0],  A_LPH[3,1], A_LPH[3,2], Qnew[3]] ])

# Flip the volume in the 2nd dimension (1 in 0-based units)
volnew = np.flip(vol, axis=1)


In [ ]:
ax = plt.figure()
slc = 15
plt.subplot(1,2,1, title='original')
plt.imshow(vol[:,:,slc])
plt.xticks([]), plt.yticks([])
plt.subplot(1,2,2, title='flipped in second dimension')
plt.imshow(volnew[:,:,slc])
plt.xticks([]), plt.yticks([])

In [ ]:
# Although flipped, we have correctly updated the geometry:

fig = plt.figure()          # Open figure and get 3D axes (can rotate with mouse)
ax  = plt.axes(projection='3d') 

vdisp(ax, volnew, Anew,  0.6, slc, 'gray')
vdisp(ax, vol,    A_LPH, 0.2, slc, 'hot')

Despite the flipped orientation in the array, the images coincide because they are the same slice and are correctly positioned in 3D space.

Possible Exercises:

Flip in the first dimension.

Apply a 90 degree rotation about the 3rd dimension axis (a simple rotation)